### TODO
1. Import data 
2. Risk index
3. Rates index
4. Position sizer
5. Theoretical positions
6. Actual futures positions
7. Theo VaR and pnl
8. Actualt VaR and pnl
9. Spread + commissions
10. Dictionary of futures specs

### Bottoms up models 
1. High frequency trend
2. ML forecasts 


### Execution system
1. HFPredictors 

### Reports
1. VaR
2. Stress test
3. Leverage 
4. Pnl 

In [ ]:
# import libraries
import pandas as pd
pd.options.display.max_columns = None 
import numpy as np
# charting libraries & set style
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn")
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
 # import our own utilities
from macro_model import *

In [ ]:
# configs
data_start_date = "2020-01-01"
analysis_start_date = "2020-06-01"
end_date = "2020-09-30"

# risk target
annual_risk_target = 0.10 # 10%
daily_risk_target = annual_risk_target / np.sqrt(252)



In [ ]:
# load the data and add models 
all_data = load_data(data_start_date, end_date, daily_risk_target)

In [ ]:
weighted_risk_index = pd.DataFrame(index = all_data)
risk_index = pd.DataFrame(index = all_data)
                                   
all_data["weighted_10y_rates_index_return"] = (all_data[["usgg10yr_index_weighted_return", 
                                                    "gecu10yr_index_weighted_return",
                                                   "gjgb10_index_weighted_return",
                                                   "gtgbp10y_govt_weighted_return",
                                                   "gacgb10_index_weighted_return",
                                                   "gcan10yr_index_weighted_return",]]).sum(axis=1)


all_data["weighted_risk_index_return"] = (all_data[["spx_index_weighted_return", 
                                                    "dax_index_weighted_return",
                                                   "nky_index_weighted_return",
                                                   "ukx_index_weighted_return",
                                                   "as51_index_weighted_return",
                                                   "sptsx_index_weighted_return",
                                                   "xau_curncy_weighted_return", 
                                                   "hg1_comdty_weighted_return"]]).sum(axis=1) 

# turn rates index the other way round
all_data["weighted_10y_rates_index_return"] = all_data["weighted_10y_rates_index_return"] * (-1)

# average of both indices 
all_data["weighted_index_return"] = all_data[["weighted_risk_index_return", 
                                                           "weighted_10y_rates_index_return"]].mean(axis=1)

all_data["weighted_index_return_20d_vol"] = all_data["weighted_index_return"].rolling(20).std() * np.sqrt(252)
all_data["scalar"] = annual_risk_target / all_data["weighted_index_return_20d_vol"]

# TODO adjust the indices for vol
# TODO correlations between risk and rates 

# cumulative returns
all_data["weighted_risk_cum_sum"] = all_data["weighted_risk_index_return"].cumsum()
all_data["weighted_rates_cum_sum"] = all_data["weighted_risk_index_return"].cumsum()



In [ ]:
# insert run model
run_model(all_data)

In [ ]:
# relative performance of risk and return
all_data["risk_30d_ma"] = all_data["weighted_risk_index_return"].rolling(21).mean()
all_data["rates_30d_ma"] = all_data["weighted_10y_rates_index_return"].rolling(21).mean()
all_data["relative_performance_30d_ma"] = (all_data["risk_30d_ma"] - all_data["rates_30d_ma"])
# create relative performance indicator
all_data["relative_performance"] = (all_data["weighted_risk_index_return"] - all_data["weighted_10y_rates_index_return"])


In [ ]:

# plot relative performance
all_data[["risk_30d_ma", 
          "rates_30d_ma", 
          "relative_performance_30d_ma"]].loc[analysis_start_date:end_date].cumsum().plot();
plt.legend();



In [ ]:
# set up signals on the weighted indices - don't shift yet 
all_data["rp_signal"] = np.where(all_data["relative_performance_30d_ma"].diff() >= 0, 1, -1)
all_data["weighted_risk_trend_signal"] = np.where((all_data["weighted_risk_cum_sum"] - all_data["weighted_risk_cum_sum"].rolling(21).mean())>=0, 1, -1)
all_data["weighted_rates_trend_signal"] = np.where((all_data["weighted_rates_cum_sum"] - all_data["weighted_rates_cum_sum"].rolling(21).mean())>=0, 1, -1)



In [ ]:
# create risk signal
all_data["risk_signal"] = 0
all_data.loc[(all_data["rp_signal"] == 1), "risk_signal"] = 1
all_data.loc[(all_data["rp_signal"] == -1) & (all_data["weighted_risk_trend_signal"] == -1), "risk_signal"] = -1
all_data.loc[(all_data["rp_signal"] == -1) & (all_data["weighted_risk_trend_signal"] == 1), "risk_signal"] = 0

# shift the signal
all_data["risk_signal"] = all_data["risk_signal"].shift(1)

In [ ]:
# create rates signal
all_data["rates_signal"] = 0
all_data.loc[(all_data["rp_signal"] == -1), "rates_signal"] = 1
all_data.loc[(all_data["rp_signal"] == 1) & (all_data["weighted_rates_trend_signal"] == -1), "rates_signal"] = -1
all_data.loc[(all_data["rp_signal"] == 1) & (all_data["weighted_rates_trend_signal"] == 1), "risk_signal"] = 1

# shift the signal
all_data["rates_signal"] = all_data["rates_signal"].shift(1)

In [ ]:
# create the risk and rates moodel pnls
all_data["risk_pnl"] = all_data["risk_signal"] * all_data["weighted_risk_index_return"]
all_data["rates_pnl"] = all_data["rates_signal"] * all_data["weighted_10y_rates_index_return"]
all_data["model_pnl"] = (all_data["risk_pnl"] + all_data["rates_pnl"])

In [ ]:
all_data[["weighted_10y_rates_index_return", "weighted_risk_index_return", "model_pnl"]].cumsum().plot()
plt.legend();

In [ ]:
ar = annual_return(all_data["model_pnl"])
risk = annual_risk(all_data["model_pnl"])
sr = sharpe_ratio(all_data["model_pnl"])
print(f"Annual return: {ar}")
print(f"Annual risk: {risk}")
print(f"Sharpe Ratio: {sr}")



In [ ]:
all_data["risk_signal"].iloc[-1]
all_data["rates_signal"].iloc[-1]


In [ ]:
output = all_data[["rp_signal", "weighted_risk_trend_signal", "risk_signal"]].tail(15).transpose()

In [ ]:
all_data[["risk_signal", "rates_signal"]].loc[analysis_start_date:end_date].plot()

In [ ]:
# create valuation indicator
all_data['risk_pct_rank'] = all_data['weighted_risk_index_return'].rolling(120).mean().rank(pct=True)
all_data['rates_pct_rank'] = all_data['weighted_10y_rates_index_return'].rolling(120).mean().rank(pct=True)
all_data["rich_cheap"] = (all_data["spx_index_60d_percentile"] + 
                          all_data["spx_index_60d_percentile_detrended"]) / 2

# create trading signal based on value
all_data['value_signal'] =  0
all_data.loc[all_data["rich_cheap"] >= .8,'value_signal'] =  -1
all_data.loc[(all_data["rich_cheap"] <= .2),'value_signal'] =  1

# shift the signal
all_data["value_signal"] = all_data["value_signal"].shift(1)

# TODO create vol signal

# TODO create momentum signal


In [ ]:
#TODO add percentile 
all_data[["spx_index", "risk_pct_rank"]].loc[analysis_start_date:end_date].plot(subplots=True)
plt.legend()
plt.title("Rich-Cheap Indicator");

In [ ]:
# TODO model for futures based on slope of this line
all_data['risk_pct_rank'].rolling(5).mean().plot();

In [ ]:
all_data['rich_cheap'].plot();

In [ ]:
all_data[['spx_index_60d_percentile', 'spx_index', 'spx_index_21d_vol']].plot(subplots=True);

In [ ]:
all_data["usgg10yr_index_weight"]*all_data["rates_signal"].tail()

In [ ]:
all_data["ty1_comdty_weight"]*all_data["rates_signal"].tail()

In [ ]:
aum = 5000000
contract_size = 100000
number_of_instruments = 6
weight = (all_data["ty1_comdty_weight"]*all_data["rates_signal"]).iloc[-1]
position = np.round((weight/number_of_instruments) * aum / contract_size)
position

In [ ]:
all_data[["ty1_comdty_weight","ty1_comdty_21d_vol", "usgg10yr_index_weight", "usgg10yr_index_21d_vol"]].tail()

In [ ]:
weight